# Character-based LSTM: Generation

In [1]:
import sys
sys.path.append('..')

import glob
import os
from functools import partial

import nltk
import numpy as np

from keras.layers import Lambda
from keras.models import clone_model

from src import models, utils

Using TensorFlow backend.


In [2]:
# Some useful parameters
KEY = 'word2vec_lstm'
DIR = os.path.join(utils.MODELS_DIR, KEY)

### Load saved data

In [3]:
# Load params.
params = utils.load_pickle(os.path.join(DIR, 'params.pkl'))
units = params['units']
size = params['size']
sg = params['sg']
iter = params['iter']
window = params['window']
window_size = params['window_size']
encoding = params['encoding']

Model = partial(models.WordLSTM, units, window_size, size)

In [4]:
# Load model with best weights
model_path = sorted(glob.glob(os.path.join(DIR, '*.h5')))[-1]
model = Model()
model.load_weights(model_path)

### Generate a sonnet

In [6]:
seed = nltk.word_tokenize('shall i compare thee to a summer\'s day?\n') + ['\n']
n_words = len(encoding.vocab)
X = np.array([encoding[word] for word in seed])
generated = seed.copy()

count = generated.count('\n')
for i in range(100):
    predicted = model.predict(np.expand_dims(X, 0)).flatten()

    # Find word that is closest to predicted encoding.
    choice = encoding.similar_by_vector(predicted, topn=1)[0][0]
    generated.append(choice)

    # "Push back" X by one and add new prediction to the end.
    X[:X.shape[0]-1] = X[1:]
    X[-1] = encoding[choice]

    count = generated.count('\n')

In [7]:
generated

['shall',
 'i',
 'compare',
 'thee',
 'to',
 'a',
 'summer',
 "'s",
 'day',
 '?',
 '\n',
 'impute',
 'sold',
 'sold',
 'thoughts',
 'confound',
 'fight',
 'instinct',
 'against',
 'come',
 'fight',
 'goodness',
 'only',
 'goodness',
 'only',
 'goodness',
 'now',
 'thousand',
 'advantage',
 'advantage',
 'advantage',
 'advantage',
 'advantage',
 'fearfully',
 'mountain',
 'on',
 'besiege',
 'fearfully',
 'oblivion',
 'same',
 'same',
 'same',
 'same',
 'same',
 'plague',
 'seeing',
 'golden',
 'seeing',
 'wretch',
 'partly',
 'laid',
 'pupil',
 'dreaming',
 'plague',
 'seeing',
 'warrantise',
 'time',
 'plague',
 'plague',
 'seeing',
 'his',
 'wretch',
 'partly',
 'instinct',
 'his',
 'audit',
 'come',
 'his',
 'doctor-like',
 'ocean',
 'quest',
 'famoused',
 'hoisted',
 'fight',
 'goodness',
 'goodness',
 'goodness',
 'goodness',
 'goodness',
 'goodness',
 'goodness',
 'his',
 'vow',
 'fight',
 'fight',
 'bare',
 'against',
 'goodness',
 'golden',
 'golden',
 'his',
 'hung',
 'robe',
 

In [6]:
print(s)

shall i compare thee to a summer's day?
thou art more lovely and more temperate:
rough winds do shake the darling buds of may,
and summer's lease hath all too short a date:
sometime too hot the eye of heaven's sun!
but you like none, in not so stould thee,
as tho eners the cunch not wail thee defeats,
though thou dost fuirts and wranting sgilled:
in praise of ladies dead, and lovely knights,
then in the blazon of sweet beauty's best,
of hand, of foot, of lip, of endlinged,
or ell-vore's deceive the dord of mine,
so then though in my hold a summer's time
to what the will come, what steals me not,



### Add temperature to the model

In [7]:
temperatures = [0.25, 0.75, 1.5]
ss = []
for t in temperatures:
    t_model = Model(temperature=t)
    t_model.load_weights(model_path)

    ss.append(t_model.generate(seed, char_to_dim, dim_to_char))

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:08<00:00,  1.66it/s]


In [8]:
for t, s in zip(temperatures, ss):
    print(f'temperature={t}')
    print(s)

temperature=0.25
shall i compare thee to a summer's day?
thou art more lovely and more temperate:
rough winds do shake the darling buds of may,
and summer's lease hath all too short a date:
sometime too hot the eye of heaven's sween.
thy love is better than high birth to me,
richer than wealth, prouder than garments' costs,
of more delight than hawks and horses be:
and having thee, of all men's pride i boast.
wretched in this alone, that thou mayst take,
all this with thee say my self i do, doth day,
that you and that which it was nourished by.
this thou perceiv'st, which makes thy love more strong,
to love that well, which thou must leave ere long.

temperature=0.75
shall i compare thee to a summer's day?
thou art more lovely and more temperate:
rough winds do shake the darling buds of may,
and summer's lease hath all too short a date:
sometime too hot the eye of heaven shines,
and often is his gold complexion dimmed,
and every fair from fair sometime declines,
by chance, or nature's 